### Look Ahead CART

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time
import tracemalloc
from queue import Queue

In [2]:
class Node:
    """Tree node for lookahead CART."""
    def __init__(self, depth, data_idx, y, parent=None, condition=None):
        self.depth = depth
        self.data_idx = data_idx
        self.y = y
        self.parent = parent
        self.condition = condition  # (feature, threshold, is_left)
        self.left = None
        self.right = None
        self.pred = self.y[data_idx].mean() if len(data_idx) > 0 else 0.0
        self.mse = np.var(self.y[data_idx]) if len(data_idx) > 0 else 0.0

class LookaheadCART:
    """CART with bounded lookahead search."""
    
    def __init__(self, max_depth=3, min_leaf_samples=10, lookahead_depth=2, n_candidate_splits=10):
        self.max_depth = max_depth
        self.min_leaf_samples = min_leaf_samples
        self.lookahead_depth = lookahead_depth
        self.n_candidate_splits = n_candidate_splits
        self.evaluations = 0
        self.pruned_branches = 0
        self.fit_time = 0
        self.peak_memory = 0
        
    def get_candidate_splits(self, X, feature):
        """Get candidate split points using quantiles."""
        vals = X[feature].values
        if len(np.unique(vals)) <= self.n_candidate_splits:
            return np.unique(vals)[:-1]  # All unique values except max
        
        quantiles = np.linspace(0.1, 0.9, self.n_candidate_splits)
        candidates = np.quantile(vals, quantiles)
        return np.unique(candidates)
    
    def calculate_mse_after_split(self, y, left_idx, right_idx):
        """Calculate weighted MSE after a split."""
        if len(left_idx) == 0 or len(right_idx) == 0:
            return float('inf')
        
        left_pred = y[left_idx].mean()
        right_pred = y[right_idx].mean()
        
        left_mse = np.mean((y[left_idx] - left_pred) ** 2)
        right_mse = np.mean((y[right_idx] - right_pred) ** 2)
        
        total_samples = len(left_idx) + len(right_idx)
        weighted_mse = (len(left_idx) * left_mse + len(right_idx) * right_mse) / total_samples
        
        return weighted_mse
    
    def find_best_split_greedy(self, X, y, data_idx):
        """Standard greedy split finding."""
        best_mse = float('inf')
        best_split = None
        
        current_mse = np.var(y[data_idx])
        
        for feature in X.columns:
            candidates = self.get_candidate_splits(X.iloc[data_idx], feature)
            
            for threshold in candidates:
                mask = X.iloc[data_idx][feature] <= threshold
                left_idx = data_idx[mask]
                right_idx = data_idx[~mask]
                
                if len(left_idx) < self.min_leaf_samples or len(right_idx) < self.min_leaf_samples:
                    continue
                
                mse = self.calculate_mse_after_split(y, left_idx, right_idx)
                self.evaluations += 1
                
                if mse < best_mse:
                    best_mse = mse
                    best_split = (feature, threshold, left_idx, right_idx)
        
        return best_split, best_mse
    
    def calculate_optimistic_bound(self, y, data_idx, remaining_depth):
        """Calculate optimistic bound on MSE reduction with perfect splits."""
        if remaining_depth <= 0 or len(data_idx) < 2 * self.min_leaf_samples:
            return np.var(y[data_idx])
        
        current_var = np.var(y[data_idx])
        # Optimistic assumption: each split level can reduce variance by 50%
        # (obviously impossible in practice, but gives us a bound)
        optimistic_var = current_var * (0.5 ** remaining_depth)
        return max(0.0, optimistic_var)
    
    def find_best_split_lookahead(self, X, y, data_idx, depth, alpha, beta, tree_depth):
        """Find best split with lookahead and alpha-beta pruning."""
        if (depth <= 0 or 
            len(data_idx) < 2 * self.min_leaf_samples or 
            tree_depth >= self.max_depth):
            return None, np.var(y[data_idx])
        
        best_split = None
        best_score = float('inf')
        
        for feature in X.columns:
            candidates = self.get_candidate_splits(X.iloc[data_idx], feature)
            
            for threshold in candidates:
                mask = X.iloc[data_idx][feature] <= threshold
                left_idx = data_idx[mask]
                right_idx = data_idx[~mask]
                
                if len(left_idx) < self.min_leaf_samples or len(right_idx) < self.min_leaf_samples:
                    continue
                
                # Immediate MSE
                immediate_mse = self.calculate_mse_after_split(y, left_idx, right_idx)
                self.evaluations += 1
                
                # Calculate total samples (needed for both lookahead and pruning)
                total_samples = len(left_idx) + len(right_idx)
                
                if depth > 1:
                    # Lookahead: evaluate future potential
                    _, left_future = self.find_best_split_lookahead(
                        X, y, left_idx, depth-1, alpha, beta, tree_depth+1)
                    _, right_future = self.find_best_split_lookahead(
                        X, y, right_idx, depth-1, alpha, beta, tree_depth+1)
                    
                    # Combine immediate and future (weighted average)
                    future_mse = (len(left_idx) * left_future + len(right_idx) * right_future) / total_samples
                    
                    # Weight immediate vs future performance
                    combined_score = 0.6 * immediate_mse + 0.4 * future_mse
                else:
                    combined_score = immediate_mse
                
                if combined_score < best_score:
                    best_score = combined_score
                    best_split = (feature, threshold, left_idx, right_idx)
                
                # Alpha-beta pruning
                beta = min(beta, combined_score)
                
                # Optimistic bound check
                left_bound = self.calculate_optimistic_bound(y, left_idx, depth-1)
                right_bound = self.calculate_optimistic_bound(y, right_idx, depth-1)
                optimistic_score = (len(left_idx) * left_bound + len(right_idx) * right_bound) / total_samples
                
                if optimistic_score >= beta or alpha >= beta:
                    self.pruned_branches += 1
                    break  # Prune remaining thresholds for this feature
            
            if alpha >= beta:
                self.pruned_branches += 1
                break  # Prune remaining features
        
        return best_split, best_score
    
    def fit(self, X, y, use_lookahead=True):
        """Fit the lookahead CART model."""
        # Start timing and memory tracking
        tracemalloc.start()
        start_time = time.perf_counter()
        
        self.evaluations = 0
        self.pruned_branches = 0
        
        # Convert to numpy if pandas Series
        if hasattr(y, 'values'):
            y = y.values
        
        # Build tree using breadth-first search
        root = Node(depth=0, data_idx=np.arange(len(y)), y=y)
        queue = Queue()
        queue.put(root)
        
        while not queue.empty():
            node = queue.get()
            
            if (node.depth >= self.max_depth or 
                len(node.data_idx) < 2 * self.min_leaf_samples):
                continue
            
            # Find best split
            if use_lookahead:
                best_split, best_score = self.find_best_split_lookahead(
                    X, y, node.data_idx, self.lookahead_depth, 
                    float('-inf'), float('inf'), node.depth)
            else:
                best_split, best_score = self.find_best_split_greedy(
                    X, y, node.data_idx)
            
            if best_split is not None:
                feature, threshold, left_idx, right_idx = best_split
                
                # Create child nodes
                left_node = Node(node.depth + 1, left_idx, y, node, (feature, threshold, True))
                right_node = Node(node.depth + 1, right_idx, y, node, (feature, threshold, False))
                
                node.left = left_node
                node.right = right_node
                
                queue.put(left_node)
                queue.put(right_node)
        
        self.tree = root
        
        # Record timing and memory
        end_time = time.perf_counter()
        current_memory, peak_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        
        self.fit_time = end_time - start_time
        self.peak_memory = peak_memory
        
        return self
    
    def predict(self, X):
        """Predict using the fitted tree."""
        def predict_row(row, node):
            while node.left is not None and node.right is not None:
                feature, threshold, _ = node.left.condition
                if row[feature] <= threshold:
                    node = node.left
                else:
                    node = node.right
            return node.pred
        
        if hasattr(X, 'iterrows'):
            return np.array([predict_row(row, self.tree) for _, row in X.iterrows()])
        else:
            return np.array([predict_row(X.iloc[i], self.tree) for i in range(len(X))])

In [3]:
def create_xor_regression_data(n=1500, noise_level=0.2, random_state=123):
    """Create XOR regression dataset like the original."""
    np.random.seed(random_state)
    
    X = pd.DataFrame({
        'X1': np.random.randint(0, 2, n),
        'X2': np.random.randint(0, 2, n), 
        'X3': np.random.randn(n),
        'X4': np.random.randn(n)
    })
    
    # XOR relationship: X1 XOR X2 plus noise
    y = ((X['X1'] ^ X['X2']) + noise_level * np.random.randn(n)).astype(float)
    
    return X, y


def create_complex_interaction_data(n=1000, random_state=42):
    """Create dataset with more complex interactions."""
    np.random.seed(random_state)
    
    X = pd.DataFrame({
        'A': np.random.uniform(-2, 2, n),
        'B': np.random.uniform(-2, 2, n),
        'C': np.random.uniform(-2, 2, n),
        'D': np.random.randn(n),
        'E': np.random.randn(n)
    })
    
    # Complex interaction: (A > 0 AND B > 0) creates different relationship with C
    y = np.zeros(n)
    for i in range(n):
        if X.iloc[i]['A'] > 0 and X.iloc[i]['B'] > 0:
            y[i] = 2.0 * X.iloc[i]['C'] + 0.1 * np.random.randn()
        elif X.iloc[i]['A'] < 0 and X.iloc[i]['B'] < 0:
            y[i] = -1.5 * X.iloc[i]['C'] + 0.1 * np.random.randn()
        else:
            y[i] = 0.5 * X.iloc[i]['C'] + 0.3 * np.random.randn()
    
    return X, y

In [4]:
def format_memory(bytes_value):
    """Format memory in human readable form."""
    if bytes_value < 1024:
        return f"{bytes_value} B"
    elif bytes_value < 1024**2:
        return f"{bytes_value/1024:.1f} KB"
    else:
        return f"{bytes_value/(1024**2):.1f} MB"


def format_time(seconds):
    """Format time in human readable form."""
    if seconds < 1e-3:
        return f"{seconds*1e6:.1f} μs"
    elif seconds < 1:
        return f"{seconds*1e3:.1f} ms"
    else:
        return f"{seconds:.3f} s"

In [5]:
def comprehensive_comparison():
    """Compare lookahead CART vs sklearn DecisionTreeRegressor baseline."""
    print("="*80)
    print("LOOKAHEAD CART vs SKLEARN BASELINE COMPARISON")
    print("="*80)
    
    datasets = [
        ("XOR Regression", lambda: create_xor_regression_data(1500, 0.2)),
        ("Complex Interactions", lambda: create_complex_interaction_data(1000)),
    ]
    
    results = []
    
    for dataset_name, dataset_func in datasets:
        print(f"\n{dataset_name}")
        print("-" * 50)
        
        # Generate data
        X, y = dataset_func()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        print(f"Train size: {len(X_train)}, Test size: {len(X_test)}, Features: {X.shape[1]}")
        
        # Sklearn DecisionTreeRegressor (Greedy Baseline)
        print("\nSklearn DecisionTreeRegressor (Greedy Baseline):")
        sklearn_tree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=10, random_state=42)
        start_time = time.perf_counter()
        sklearn_tree.fit(X_train, y_train)
        sklearn_time = time.perf_counter() - start_time
        
        y_pred_sklearn = sklearn_tree.predict(X_test)
        mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
        
        print(f"  MSE: {mse_sklearn:.4f}")
        print(f"  Time: {format_time(sklearn_time)}")
        
        # Our Lookahead CART
        print("\nLookahead CART (depth=2):")
        lookahead_cart = LookaheadCART(max_depth=3, lookahead_depth=2, min_leaf_samples=10)
        lookahead_cart.fit(X_train, y_train, use_lookahead=True)
        
        y_pred_lookahead = lookahead_cart.predict(X_test)
        mse_lookahead = mean_squared_error(y_test, y_pred_lookahead)
        
        print(f"  MSE: {mse_lookahead:.4f}")
        print(f"  Time: {format_time(lookahead_cart.fit_time)}")
        print(f"  Memory: {format_memory(lookahead_cart.peak_memory)}")
        print(f"  Evaluations: {lookahead_cart.evaluations}")
        print(f"  Pruned branches: {lookahead_cart.pruned_branches}")
        
        # Comparison
        mse_improvement = mse_sklearn - mse_lookahead
        time_ratio = lookahead_cart.fit_time / sklearn_time if sklearn_time > 0 else 1.0
        pruning_rate = lookahead_cart.pruned_branches / (lookahead_cart.evaluations + lookahead_cart.pruned_branches) if (lookahead_cart.evaluations + lookahead_cart.pruned_branches) > 0 else 0
        
        print(f"\nComparison vs Sklearn:")
        print(f"  MSE improvement: {mse_improvement:.4f}")
        print(f"  Time ratio: {time_ratio:.2f}x")
        print(f"  Pruning effectiveness: {pruning_rate:.1%}")
        
        if mse_improvement > 0.01:
            print(f"  ✅ Lookahead provides significant improvement!")
        elif mse_improvement > 0.001:
            print(f"  ≈ Modest improvement")
        else:
            print(f"  ⚠️ No significant improvement over sklearn")
        
        results.append({
            'dataset': dataset_name,
            'mse_sklearn': mse_sklearn,
            'mse_lookahead': mse_lookahead,
            'improvement': mse_improvement,
            'time_ratio': time_ratio,
            'pruning_rate': pruning_rate,
            'evaluations': lookahead_cart.evaluations,
            'pruned_branches': lookahead_cart.pruned_branches
        })
    
    # Summary
    print(f"\n{'='*80}")
    print("SUMMARY")
    print(f"{'='*80}")
    
    avg_improvement = np.mean([r['improvement'] for r in results])
    avg_time_ratio = np.mean([r['time_ratio'] for r in results])
    avg_pruning_rate = np.mean([r['pruning_rate'] for r in results])
    total_evaluations = sum([r['evaluations'] for r in results])
    total_pruned = sum([r['pruned_branches'] for r in results])
    
    print(f"Average MSE improvement vs Sklearn: {avg_improvement:.4f}")
    print(f"Average time overhead: {avg_time_ratio:.2f}x")
    print(f"Average pruning rate: {avg_pruning_rate:.1%}")
    print(f"Total evaluations: {total_evaluations}")
    print(f"Total branches pruned: {total_pruned}")
    
    # Best case analysis
    best_dataset = max(results, key=lambda x: x['improvement'])
    print(f"\nLookahead most effective on: {best_dataset['dataset']}")
    print(f"  MSE improvement: {best_dataset['improvement']:.4f}")
    print(f"  Time overhead: {best_dataset['time_ratio']:.2f}x")
    
    return results


if __name__ == "__main__":
    results = comprehensive_comparison()

LOOKAHEAD CART vs SKLEARN BASELINE COMPARISON

XOR Regression
--------------------------------------------------
Train size: 1050, Test size: 450, Features: 4

Sklearn DecisionTreeRegressor (Greedy Baseline):
  MSE: 0.3157
  Time: 2.1 ms

Lookahead CART (depth=2):
  MSE: 0.0397
  Time: 376.8 ms
  Memory: 139.4 KB
  Evaluations: 312
  Pruned branches: 208

Comparison vs Sklearn:
  MSE improvement: 0.2759
  Time ratio: 178.90x
  Pruning effectiveness: 40.0%
  ✅ Lookahead provides significant improvement!

Complex Interactions
--------------------------------------------------
Train size: 700, Test size: 300, Features: 5

Sklearn DecisionTreeRegressor (Greedy Baseline):
  MSE: 0.6408
  Time: 1.9 ms

Lookahead CART (depth=2):
  MSE: 0.7151
  Time: 2.283 s
  Memory: 102.7 KB
  Evaluations: 1772
  Pruned branches: 1510

Comparison vs Sklearn:
  MSE improvement: -0.0742
  Time ratio: 1223.84x
  Pruning effectiveness: 46.0%
  ⚠️ No significant improvement over sklearn

SUMMARY
Average MSE impr